In [ ]:
import numpy as np
import tensorflow as tf
import utils.socket_util as su
import yfinance as yf
import cufflinks as cf
import matplotlib.pyplot as plt
import db.raw_k_db as k
import indicators.atr as atr
import indicators.advantage_excursion as ae
import db.codes_db as cdb
import datetime
import random

su.set_proxy()
cf.set_config_file(offline=True)

In [ ]:
codes = cdb.find_all_code()
random.shuffle(codes)
sp = int(len(codes)*0.7)

t_codes = codes[0:sp]
v_codes = codes[sp:len(codes)]
print(t_codes, v_codes)

In [ ]:
np.savez('training_codes', codes=np.asarray(t_codes))
np.savez('validation_codes', codes=np.asarray(v_codes))

In [ ]:
samples_t = ae.build_samples(t_codes)

In [ ]:
validation_codes = np.load('validation_codes.npz')
v_codes = validation_codes['codes']

In [ ]:
samples_v = ae.build_samples(v_codes)

In [ ]:
np.savez('validation_raw_k', k=ae.training_samples_by(samples_v, 'k'))

In [ ]:
np.savez('validation_y', label_bull_hot=ae.training_samples_by(samples_v, 'label_bull_hot'), label_bear_hot=ae.training_samples_by(samples_v, 'label_bear_hot'), label_total_hot=ae.training_samples_by(samples_v, 'label_total_hot'))

In [ ]:
np.savez('training_raw_data', k=ae.training_samples_by(samples_t, 'k'), label_bull_hot=ae.training_samples_by(samples_t, 'label_bull_hot'), label_bear_hot=ae.training_samples_by(samples_t, 'label_bear_hot'), label_total_hot=ae.training_samples_by(samples_t, 'label_total_hot'))

In [ ]:
training_raw_data = np.load('training_raw_data.npz')

In [ ]:
np.savez('training_y', label_bull_hot=training_raw_data['label_bull_hot'], label_bear_hot=training_raw_data['label_bear_hot'], label_total_hot=training_raw_data['label_total_hot'])

In [ ]:
np.savez('training_raw_k', k=training_raw_data['k'])

In [ ]:
training_raw_k = np.load('training_raw_k.npz')

In [ ]:
normalization_samples_k = ae.normalization(training_raw_k['k'])

In [ ]:
np.savez('training_nor_k', k=normalization_samples_k)

In [ ]:
validation_raw_k = np.load('validation_raw_k.npz')

In [ ]:
normalization_samples_k_v = ae.normalization(validation_raw_k['k'])

In [ ]:
np.savez('validation_nor_k', k=normalization_samples_k_v)

In [ ]:
normalization_samples_k_v

In [ ]:
spy = k.find_by_code_and_interval('SPY', '1d')
spy = atr.calculate(spy)
spy = ae.calculate(spy)
spy = ae.mark(spy)
samples_v = ae.get_training_samples(spy)

In [ ]:
model = tf.keras.models.Sequential([
    # input_shape 保持和数据集一致
    tf.keras.layers.Dense(32,input_shape=(120, 4), activation='relu'),
    # tf.keras.layers.MaxPool1D(2),

    tf.keras.layers.Dense(64, activation='relu'),
    # tf.keras.layers.MaxPool1D(2),

    tf.keras.layers.Dense(128,activation='relu'),
    # tf.keras.layers.MaxPool1D(2),

    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(9, activation='sigmoid'),
])
model.build()
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['acc'])

In [ ]:
his = model.fit(ae.training_samples_by(samples_t,'k'), ae.training_samples_by(samples_t,'label_bull_hot'), epochs=20, validation_data=(ae.training_samples_by(samples_v,'k'), ae.training_samples_by(samples_v,'label_bull_hot')))

In [ ]:
plt.plot(his.history['acc'], label = 'Training')
plt.plot(his.history['val_acc'], label = 'Validation')
plt.legend()
plt.show()

In [ ]:
xlp = k.find_by_code_and_interval('XLP', '1d')
xlp = xlp.loc[xlp.query('low > 2')['low'].idxmin():xlp.index[-1]] if xlp.loc[xlp.index[0]]['low'] < 2 else xlp
xlp = atr.calculate(xlp)
xlp = ae.calculate(xlp)
xlp = ae.mark(xlp)

samples = ae.get_training_samples(xlp)
k_hots = ae.training_samples_by(samples, 'k')

print(k_hots)

In [ ]:
a = [1]
b = [0]
a.extend(b)

In [ ]:
a